### Import Required Libraries and Set Up Environment Variables

In [101]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime

## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

# Confirm success
if NASA_API_KEY:
    print(f"NASA API Key loaded: {NASA_API_KEY}")
else:
    print("Error: NASA API Key not found. Check your .env file.")

# Adjust display settings to avoid truncation and increase width
pd.set_option('display.max_colwidth', None)  # Show full contents of columns without truncation
pd.set_option('display.width', 1000)  # Increase the width of the display to avoid wrapping

NASA API Key loaded: R1YaLwmk3dKS5mnxlnnZADyr86WmGO6Xtow7OMMQ


### CME Data

In [102]:
# Set the base URL to NASA's DONKI API: 
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
specifer = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
query_url = F"{base_url}{specifer}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

print(f"Constructed query URL: {query_url}")

Constructed query URL: https://api.nasa.gov/DONKI/CME?startDate=2013-05-01&endDate=2024-05-01&api_key=R1YaLwmk3dKS5mnxlnnZADyr86WmGO6Xtow7OMMQ


In [103]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response 
cme_response = requests.get(query_url)

#check success
if cme_response.status_code == 200:
    print("CME data retreieved succussfully!")
else:
    print(f"Failed to retrieve CME data. Status code: {cme_response.status_code}")


CME data retreieved succussfully!


In [104]:
# Convert the response variable to json and store it as a variable named cme_json 
cme_json = cme_response.json()

print(cme_json[:3])

[{'activityID': '2013-05-01T03:12:00-CME-001', 'catalog': 'M2M_CATALOG', 'startTime': '2013-05-01T03:12Z', 'instruments': [{'displayName': 'SOHO: LASCO/C2'}, {'displayName': 'SOHO: LASCO/C3'}, {'displayName': 'STEREO A: SECCHI/COR2'}, {'displayName': 'STEREO B: SECCHI/COR2'}], 'sourceLocation': '', 'activeRegionNum': None, 'note': '', 'submissionTime': '2013-08-07T16:54Z', 'versionId': 1, 'link': 'https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1', 'cmeAnalyses': [{'isMostAccurate': True, 'time21_5': '2013-05-01T07:07Z', 'latitude': 12.0, 'longitude': -120.0, 'halfAngle': 36.0, 'speed': 860.0, 'type': 'C', 'featureCode': 'null', 'imageType': None, 'measurementTechnique': 'null', 'note': '', 'levelOfData': 0, 'tilt': None, 'minorHalfWidth': None, 'speedMeasuredAtHeight': None, 'submissionTime': '2013-08-07T16:54Z', 'link': 'https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CMEAnalysis/2350/-1', 'enlilList': [{'modelCompletionTime': '2013-05-01T08:32Z', 'au': 2.0, 'estimatedShockAr

In [105]:
# Preview the first result in JSON format 
# Use json.dumps with argument indent=4 to format data
if cme_json:
    first_result = cme_json[0]
    formatted_first_result = json.dumps(first_result, indent=4)
    print(formatted_first_result)
else:
    print("No CME Data available to preview.")


{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [106]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
cme_filtered_df = cme_df[['activityID', 'startTime', 'linkedEvents']]

print(cme_filtered_df.head())

                    activityID          startTime                                     linkedEvents
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z  [{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1  2013-05-02T05:24:00-CME-001  2013-05-02T05:24Z                                             None
2  2013-05-02T14:36:00-CME-001  2013-05-02T14:36Z                                             None
3  2013-05-03T18:00:00-CME-001  2013-05-03T18:00Z                                             None
4  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z  [{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [107]:
# Notice that the linkedEvents column allows us to identify the corresponding GST 
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_filtered_df_cleaned = cme_filtered_df.dropna(subset=['linkedEvents'])

print(cme_filtered_df_cleaned.head())

                     activityID          startTime                                                                                    linkedEvents
0   2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z                                                 [{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4   2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z                                                 [{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7   2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z                                                 [{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z  [{'activityID': '2013-05-13T01:53:00-FLR-001'}, {'activityID': '2013-05-13T04:12:00-SEP-001'}]
13  2013-05-13T16:18:00-CME-001  2013-05-13T16:18Z  [{'activityID': '2013-05-13T15:40:00-FLR-001'}, {'activityID': '2013-05-13T18:02:00-SEP-001'}]


In [108]:
# Notice that the linkedEvents sometimes contains multiple events per row 
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme_filtered_df_cleaned.index:
    activity_id = cme_filtered_df_cleaned.loc[i, 'activityID']
    start_time = cme_filtered_df_cleaned.loc[i, 'startTime']
    linked_events = cme_filtered_df_cleaned.loc[i, 'linkedEvents']

    # Iterate over each dictionary in the list
    if isinstance(linked_events, list):
        for event in linked_events:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
            expanded_rows.append({
                'activityID': activity_id,
                'startTime': start_time,
                'linkedEventActivityID': event['activityID']
        })
             
# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

print(expanded_df.head())

                    activityID          startTime        linkedEventActivityID
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z  2013-05-04T04:52:00-IPS-001
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z  2013-05-07T04:37:00-IPS-001
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z  2013-05-12T23:30:00-IPS-001
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z  2013-05-13T01:53:00-FLR-001
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z  2013-05-13T04:12:00-SEP-001


In [109]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents 
def extract_activityID_from_dict(input_dict):
    try:
        if isinstance(input_dict, list) and len(input_dict) > 0 and 'activityID' in input_dict[0]:
            return input_dict[0]['activityID']
        else:
            return None
    except (ValueError, TypeError, KeyError) as e:
        print(f"Error extracting activityID: {e}")
        return None
# and verify below that it works as expected using one row from linkedEvents as an example
example_linked_event = cme_filtered_df_cleaned.loc[0, 'linkedEvents']
extracted_activity_id = extract_activityID_from_dict(example_linked_event)

# Be sure to use a try and except block to handle errors

        # Log the error or print it for debugging
print(f"Extracted activityID: {extracted_activity_id}")



Extracted activityID: 2013-05-04T04:52:00-IPS-001


In [110]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function) 
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_filtered_df_cleaned = cme_filtered_df_cleaned.copy()
cme_filtered_df_cleaned.loc[:, 'GST_ActivityID'] = cme_filtered_df_cleaned['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Display the updated DataFrame to verify the new column
print(cme_filtered_df_cleaned[['activityID', 'GST_ActivityID']].head())



                     activityID               GST_ActivityID
0   2013-05-01T03:12:00-CME-001  2013-05-04T04:52:00-IPS-001
4   2013-05-03T22:36:00-CME-001  2013-05-07T04:37:00-IPS-001
7   2013-05-09T19:29:00-CME-001  2013-05-12T23:30:00-IPS-001
10  2013-05-13T02:54:00-CME-001  2013-05-13T01:53:00-FLR-001
13  2013-05-13T16:18:00-CME-001  2013-05-13T15:40:00-FLR-001


In [111]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs: 
cme_final_df = cme_filtered_df_cleaned.dropna(subset=['GST_ActivityID'])

print(cme_final_df[['activityID', 'startTime', 'GST_ActivityID']].head())

                     activityID          startTime               GST_ActivityID
0   2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z  2013-05-04T04:52:00-IPS-001
4   2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z  2013-05-07T04:37:00-IPS-001
7   2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z  2013-05-12T23:30:00-IPS-001
10  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z  2013-05-13T01:53:00-FLR-001
13  2013-05-13T16:18:00-CME-001  2013-05-13T16:18Z  2013-05-13T15:40:00-FLR-001


In [112]:
# print out the datatype of each column in this DataFrame: 
print(cme_final_df.dtypes)


activityID        object
startTime         object
linkedEvents      object
GST_ActivityID    object
dtype: object


In [113]:
# Convert the 'GST_ActivityID' column to string format 
cme_final_df['GST_ActivityID'] = cme_final_df['GST_ActivityID'].astype('string')

# Convert startTime to datetime format  
cme_final_df['startTime'] = pd.to_datetime(cme_final_df['startTime'], utc=True)

# Rename startTime to startTime_CME and activityID to cmeID
cme_final_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop linkedEvents
if 'linkedEvents' in cme_final_df.columns:
    cme_final_df.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
print(cme_final_df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 1023 entries, 0 to 5523
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1023 non-null   object             
 1   startTime_CME   1023 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1023 non-null   string             
dtypes: datetime64[ns, UTC](1), object(1), string(1)
memory usage: 64.3+ KB
None


In [114]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST' 
# use the method 'contains()' from the str library.  
cme_gst_df = cme_final_df[cme_final_df['GST_ActivityID'].str.contains('GST', na=False)]

print(cme_gst_df.head())


                            cmeID             startTime_CME               GST_ActivityID
26    2013-06-02T20:24:00-CME-001 2013-06-02 20:24:00+00:00  2013-06-07T03:00:00-GST-001
1358  2015-09-04T14:12:00-CME-001 2015-09-04 14:12:00+00:00  2015-09-09T03:00:00-GST-001
1359  2015-09-04T19:24:00-CME-001 2015-09-04 19:24:00+00:00  2015-09-09T03:00:00-GST-001
1361  2015-09-06T23:30:00-CME-001 2015-09-06 23:30:00+00:00  2015-09-11T06:00:00-GST-001
1362  2015-09-07T10:36:00-CME-001 2015-09-07 10:36:00+00:00  2015-09-11T06:00:00-GST-001


### GST Data

In [115]:
# Set the base URL to NASA's DONKI API: 
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
query_url_gst = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

print(f"Constructed GST URL: {query_url_gst}")

Constructed GST URL: https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2024-05-01&api_key=R1YaLwmk3dKS5mnxlnnZADyr86WmGO6Xtow7OMMQ


In [116]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response 
gst_response = requests.get(query_url_gst)

# check success
if gst_response.status_code == 200:
    print("GST data retrieved successfully!")
    gst_json = gst_response.json()
else:
    print(f"Failed to retrieve GST data. Status code: {gst_response.status_code}")


GST data retrieved successfully!


In [117]:
# Convert the response variable to json and store it as a variable named gst_json 
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(gst_json[0], indent=4))


{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [118]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents
gst_filtered_df = gst_df[['gstID', 'startTime', 'linkedEvents']].copy()

gst_filtered_df['activityID'] = gst_filtered_df['linkedEvents'].apply(lambda x: x[0]['activityID'] if x and isinstance(x, list) else None)

print(gst_filtered_df[['gstID', 'startTime', 'activityID']].head())

                         gstID          startTime                   activityID
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z  2013-05-31T15:45:00-HSS-001
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z  2013-06-02T20:24:00-CME-001
2  2013-06-29T03:00:00-GST-001  2013-06-29T03:00Z                         None
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z  2013-09-29T22:40:00-CME-001
4  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z  2013-12-04T23:12:00-CME-001


In [119]:
# Notice that the linkedEvents column allows us to identify the corresponding CME 
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_filtered_df_cleaned = gst_filtered_df.dropna(subset=['linkedEvents'])

print(gst_filtered_df_cleaned[['gstID', 'startTime', 'activityID']].head())

                         gstID          startTime                   activityID
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z  2013-05-31T15:45:00-HSS-001
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z  2013-06-02T20:24:00-CME-001
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z  2013-09-29T22:40:00-CME-001
4  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z  2013-12-04T23:12:00-CME-001
5  2014-02-19T03:00:00-GST-001  2014-02-19T03:00Z  2014-02-16T14:15:00-CME-001


In [120]:
# Notice that the linkedEvents sometimes contains multiple events per row 
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_exploded_df = gst_filtered_df_cleaned.explode('linkedEvents').reset_index(drop=True)

print(gst_exploded_df[['gstID', 'startTime', 'linkedEvents']].head())

                         gstID          startTime                                   linkedEvents
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z  {'activityID': '2013-05-31T15:45:00-HSS-001'}
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z  {'activityID': '2013-06-02T20:24:00-CME-001'}
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z  {'activityID': '2013-09-29T22:40:00-CME-001'}
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z  {'activityID': '2013-10-02T01:54:00-IPS-001'}
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z  {'activityID': '2013-10-02T02:47:00-MPC-001'}


In [121]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function) 
# and create a new column called 'CME_ActivityID' using loc indexer:
def extract_activityID_from_dict(event):
    if isinstance(event, dict) and 'activityID' in event:
        return event ['activityID']
    return None

gst_exploded_df.loc[:, 'CME_ActivityID'] = gst_exploded_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_exploded_df_cleaned = gst_exploded_df.dropna(subset=['CME_ActivityID'])

print(gst_exploded_df_cleaned[['gstID', 'startTime', 'linkedEvents', 'CME_ActivityID']].head())


                         gstID          startTime                                   linkedEvents               CME_ActivityID
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z  {'activityID': '2013-05-31T15:45:00-HSS-001'}  2013-05-31T15:45:00-HSS-001
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z  {'activityID': '2013-06-02T20:24:00-CME-001'}  2013-06-02T20:24:00-CME-001
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z  {'activityID': '2013-09-29T22:40:00-CME-001'}  2013-09-29T22:40:00-CME-001
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z  {'activityID': '2013-10-02T01:54:00-IPS-001'}  2013-10-02T01:54:00-IPS-001
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z  {'activityID': '2013-10-02T02:47:00-MPC-001'}  2013-10-02T02:47:00-MPC-001


In [122]:
# Convert the 'CME_ActivityID' column to string format 
gst_exploded_df_cleaned['CME_activityID'] = gst_exploded_df_cleaned['CME_ActivityID'].astype('string')

# Convert the 'gstID' column to string format 
gst_exploded_df_cleaned['gstID'] = gst_exploded_df_cleaned['gstID'].astype('string')

# Convert startTime to datetime format  
gst_exploded_df_cleaned['startTime'] = pd.to_datetime(gst_exploded_df_cleaned['startTime'], utc=True)

# Rename startTime to startTime_GST 
gst_exploded_df_cleaned.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop linkedEvents
gst_exploded_df_cleaned.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
print(gst_exploded_df_cleaned.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    string             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   activityID      205 non-null    object             
 3   CME_ActivityID  205 non-null    object             
 4   CME_activityID  205 non-null    string             
dtypes: datetime64[ns, UTC](1), object(2), string(2)
memory usage: 8.1+ KB
None


In [123]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME' 
# use the method 'contains()' from the str library.  
gst_cme_filtered_df = gst_exploded_df_cleaned[gst_exploded_df_cleaned['CME_ActivityID'].str.contains('CME')]

print(gst_cme_filtered_df[['gstID', 'startTime_GST', 'CME_activityID']].head())

                         gstID             startTime_GST               CME_activityID
1  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00  2013-06-02T20:24:00-CME-001
2  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00  2013-09-29T22:40:00-CME-001
5  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00  2013-12-04T23:12:00-CME-001
7  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00  2014-02-16T14:15:00-CME-001
9  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00  2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [124]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers. 
merged_df = pd.merge(gst_cme_filtered_df, cme_df, left_on='CME_ActivityID', right_on='activityID', how='inner')

print(merged_df[['gstID', 'startTime_GST', 'CME_ActivityID', 'activityID_y', 'startTime']].head())

                         gstID             startTime_GST               CME_ActivityID                 activityID_y          startTime
0  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00  2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001  2013-06-02T20:24Z
1  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00  2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001  2013-09-29T22:40Z
2  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00  2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001  2013-12-04T23:12Z
3  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00  2014-02-16T14:15:00-CME-001  2014-02-16T14:15:00-CME-001  2014-02-16T14:15Z
4  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00  2014-02-18T01:25:00-CME-001  2014-02-18T01:25:00-CME-001  2014-02-18T01:25Z


In [135]:
# Define final_df using the correct column names 
final_df = merged_df[['gstID', 'startTime_GST', 'CME_ActivityID', 'activityID_y', 'startTime']]

# Rename columns for clarity without using inplace to avoid the warning
final_df = final_df.rename(columns={'CME_ActivityID': 'GST_ActivityID', 'activityID_y': 'cmeID', 'startTime': 'startTime_CME'})

# Verify the number of rows in the new DataFrame
print(f"Number of rows in the merged DataFrame: {len(final_df)}")
print(f"Number of rows in the GST DataFrame: {len(gst_cme_filtered_df)}")
print(f"Number of rows in the CME DataFrame: {len(cme_df)}")

# Verify the structure and datatypes of the new DataFrame
print(final_df.info())


Number of rows in the merged DataFrame: 61
Number of rows in the GST DataFrame: 61
Number of rows in the CME DataFrame: 5524
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           61 non-null     string             
 1   startTime_GST   61 non-null     datetime64[ns, UTC]
 2   GST_ActivityID  61 non-null     object             
 3   cmeID           61 non-null     object             
 4   startTime_CME   61 non-null     object             
dtypes: datetime64[ns, UTC](1), object(3), string(1)
memory usage: 2.5+ KB
None


### Computing the time it takes for a CME to cause a GST

In [137]:
# Convert 'startTime_CME' to datetime format if it's not already 
final_df['startTime_CME'] = pd.to_datetime(final_df['startTime_CME'], errors='coerce', utc=True)

# Compute the time difference between startTime_GST and startTime_CME
final_df['timeDiff'] = final_df['startTime_GST'] - final_df['startTime_CME']

# Display the resulting DataFrame to verify the timeDiff column and ensure all necessary columns are present
print(final_df[['gstID', 'startTime_GST', 'GST_ActivityID', 'cmeID', 'startTime_CME', 'timeDiff']].head())



                         gstID             startTime_GST               GST_ActivityID                        cmeID             startTime_CME        timeDiff
0  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00  2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001 2013-06-02 20:24:00+00:00 4 days 06:36:00
1  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00  2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001 2013-09-29 22:40:00+00:00 2 days 04:20:00
2  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00  2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001 2013-12-04 23:12:00+00:00 3 days 00:48:00
3  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00  2014-02-16T14:15:00-CME-001  2014-02-16T14:15:00-CME-001 2014-02-16 14:15:00+00:00 2 days 12:45:00
4  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00  2014-02-18T01:25:00-CME-001  2014-02-18T01:25:00-CME-001 2014-02-18 01:25:00+00:00 2 days 01:35:00


In [139]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
time_stats = final_df['timeDiff'].describe()

print(time_stats)

count                           61
mean     2 days 23:29:26.557377049
std      0 days 23:53:09.336914240
min                1 days 08:36:00
25%                2 days 04:00:00
50%                2 days 19:51:00
75%                3 days 13:35:00
max                6 days 03:00:00
Name: timeDiff, dtype: object


### Exporting data in csv format

In [140]:
# Export data to CSV without the index 
final_df.to_csv('cme_gst_data.csv', index=False)

print("Data exported successfully to 'cme_gst_data.csv'")

Data exported successfully to 'cme_gst_data.csv'
